In [1]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips, get_pixel_growth_and_new_children
from subprocess import call

In [26]:
from directory import directory
# directory = "/scratch/shared/mrozemul/Fiji.app/"

plate = 40
dates_datetime = get_dates_datetime(directory,plate)
begin = 0
end = 19
print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])

begin = 2020-11-19 20:21:00 
  end = 2020-11-23 00:21:00


In [28]:
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp = pickle.load( open(f'{directory}/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

In [29]:
growth = []
select_hyph = []
for hyph in exp.hyphaes:
    for i,t in enumerate(hyph.ts[:-1]):
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,hyph.ts[i+1])
        growth.append(pixels)
        length = np.sum([len(seg) for seg in pixels])
        select_hyph.append((hyph,t,hyph.ts[i+1],length))
        

In [68]:
overgrowing = [c for c in select_hyph if c[3]>=20 and c[3]<=100]
    

In [71]:
hyph,t,tp1,le = choice(overgrowing)
print(le)

45


In [72]:
plt.close('all')
exp.plot([t,tp1],[[hyph.end.label,hyph.root.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
hyph.end.show_source_image(t,tp1)

OSError: The file /scratch/shared/AMF914/Fiji.app/20201121_0421_Plate40/Analysis/skeleton_realigned.mat does not exist.

In [5]:
growth_length = [np.sum([len(seg) for seg in segs]) for segs in growth]

In [33]:
selected_growth = [c[3] for c in select_hyph if c[3]>50]

In [34]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([188., 118.,  70.,  43.,  27.,  30.,  26.,  22.,  23.,  21.,  23.,
         32.,  48.,  43.,  40.,  17.,   0.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  51.        ,   95.76666667,  140.53333333,  185.3       ,
         230.06666667,  274.83333333,  319.6       ,  364.36666667,
         409.13333333,  453.9       ,  498.66666667,  543.43333333,
         588.2       ,  632.96666667,  677.73333333,  722.5       ,
         767.26666667,  812.03333333,  856.8       ,  901.56666667,
         946.33333333,  991.1       , 1035.86666667, 1080.63333333,
        1125.4       , 1170.16666667, 1214.93333333, 1259.7       ,
        1304.46666667, 1349.23333333, 1394.        ]),
 <a list of 30 Patch objects>)

In [15]:
np.mean(branch_frequ)

0.021061224646895957